# Manual Recheck 

## complemented for manualcheck.py

In [1]:
import glob
import matplotlib.pyplot as plt

In [2]:
match_base = './io/match/'
image_base = './io/image/'
hst_base = match_base

## Processing which image?

In [3]:
img_name = 'nircam1'

## MatchCat will help

In [4]:
from utils.matchcat import MatchCat

In [5]:
hst_dir = hst_base + img_name +'/'
hst_path = glob.glob(hst_dir + 'egs*.fits')[0]
matc = MatchCat(img_name)

## Interactive checking

In [6]:
from ipywidgets import interact, widgets

In [7]:
def rand_cmap(nlabels, type='bright', first_color_black=True, last_color_black=False, verbose=True):
    """
    Creates a random colormap to be used together with matplotlib. Useful for segmentation tasks
    :param nlabels: Number of labels (size of colormap)
    :param type: 'bright' for strong colors, 'soft' for pastel colors
    :param first_color_black: Option to use first color as black, True or False
    :param last_color_black: Option to use last color as black, True or False
    :param verbose: Prints the number of labels and shows the colormap. True or False
    :return: colormap for matplotlib
    """
    from matplotlib.colors import LinearSegmentedColormap
    import colorsys
    import numpy as np


    if type not in ('bright', 'soft'):
        print ('Please choose "bright" or "soft" for type')
        return

    if verbose:
        print('Number of labels: ' + str(nlabels))

    # Generate color map for bright colors, based on hsv
    if type == 'bright':
        randHSVcolors = [(np.random.uniform(low=0.0, high=1),
                          np.random.uniform(low=0.2, high=1),
                          np.random.uniform(low=0.9, high=1)) for i in range(nlabels)]

        # Convert HSV list to RGB
        randRGBcolors = []
        for HSVcolor in randHSVcolors:
            randRGBcolors.append(colorsys.hsv_to_rgb(HSVcolor[0], HSVcolor[1], HSVcolor[2]))

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]

        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Generate soft pastel colors, by limiting the RGB spectrum
    if type == 'soft':
        low = 0.6
        high = 0.95
        randRGBcolors = [(np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high)) for i in range(nlabels)]

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]
        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Display colorbar
    if verbose:
        from matplotlib import colors, colorbar
        from matplotlib import pyplot as plt
        fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))

        bounds = np.linspace(0, nlabels, nlabels + 1)
        norm = colors.BoundaryNorm(bounds, nlabels)

        cb = colorbar.ColorbarBase(ax, cmap=random_colormap, norm=norm, spacing='proportional', ticks=None,
                                   boundaries=bounds, format='%1i', orientation=u'horizontal')

    return random_colormap

In [8]:
new_cmap = rand_cmap(50000, type='bright', first_color_black=True, last_color_black=False, verbose=False)
def plot_image(seq,cmap,vmin,vmax,norm):
    hst_cutout, jwst_cutout, segm_cutout = matc.get_image(hst_path,int(seq))
    fig, ax = plt.subplots(1,3, figsize=(12,4))
    ax[0].imshow(hst_cutout, origin='lower', cmap=cmap, vmin=vmin, vmax=vmax, norm=norm)
    ax[0].set_title('HST')
    ax[1].imshow(jwst_cutout, origin='lower', cmap=cmap, vmin=vmin*10, vmax=vmax*10, norm=norm)
    ax[1].set_title('JWST')
    ax[2].imshow(segm_cutout, origin='lower', cmap=new_cmap)
    ax[2].set_title('Segmentation')
    plt.show()

In [9]:
# Create a slider widget
cmap_dropdown = widgets.Dropdown(options=['viridis','gray','hot','jet','binary'],value='viridis',description='cmap')
vmin_slider = widgets.FloatSlider(min=0., max=0.002, step=.00002, value=0.)
vmax_slider = widgets.FloatSlider(min=0.002, max=0.02, step=.0002, value=.01)
norm_dropdown = widgets.Dropdown(options=['linear','log','asinh'],value='linear',description='norm')
i_text = widgets.Text(value='0',description='seq')

# Use the interact function to update the plot when the slider is changed
_=interact(plot_image, seq=i_text,cmap=cmap_dropdown,vmin=vmin_slider,vmax=vmax_slider,norm=norm_dropdown)

interactive(children=(Text(value='0', description='seq'), Dropdown(description='cmap', options=('viridis', 'gr…